# Introduction

One of the most fundamental tasks during a data analysis involves splitting data into independent groups before performing a calculation on each group. This methodology has been around for quite some time but has more recently been referred to as split-apply-combine. This chapter covers the powerful groupby method, which allows you to group your data in any way imaginable and apply any type of function independently to each group before returning a single dataset.

**Note**

Hadley Wickham coined the term split-apply-combine to describe the common data analysis pattern of breaking up data into independent manageable chunks, independently applying functions to these chunks, and then combining the results back together. More details can be found in his paper (http://bit.ly/2isFuL9).

Before we get started with the recipes, we will need to know just a little terminology. All basic groupby operations have grouping columns, and each unique combination of values in these columns represents an independent grouping of the data. The syntax looks as follows:

`df.groupby(['list', 'of', 'grouping', 'columns'])
df.groupby('single_column')  # when grouping by a single column`

The result of this operation returns a groupby object. It is this groupby object that will be the engine that drives all the calculations for this entire chapter. Pandas actually does very little when creating this groupby object, merely validating that grouping is possible. You will have to chain methods on this groupby object in order to unleash its powers.

**Note**

Technically, the result of the operation will either be a DataFrameGroupBy or SeriesGroupBy but for simplicity, it will be referred to as the groupby object for the entire chapter.



# Defining an aggregation

The most common use of the groupby method is to perform an aggregation. What actually is an aggregation? In our data analysis world, an aggregation takes place when a sequence of many inputs get summarized or combined into a single value output. For example, summing up all the values of a column or finding its maximum are common aggregations applied on a single sequence of data. An aggregation simply takes many values and converts them down to a single value.

In addition to the grouping columns defined during the introduction, most aggregations have two other components, the aggregating columns and aggregating functions. The aggregating columns are those whose values will be aggregated. `The aggregating functions define how the aggregation takes place. Major aggregation functions include sum, min, max, mean, count, variance, std, and so on.`

### Getting ready

In this recipe, we examine the flights dataset and perform the simplest possible aggregation involving only a single grouping column, a single aggregating column, and a single aggregating function. `We will find the average arrival delay for each airline. Pandas has quite a few different syntaxes to produce an aggregation and this recipe covers them.`

### How to do it...

Read in the flights dataset, and define the grouping columns (AIRLINE), aggregating columns (ARR_DELAY), and aggregating functions (mean):

In [1]:
import pandas as pd
import numpy as np

In [2]:
flights = pd.read_csv('data/flights.csv')
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


Place the grouping column in the groupby method and then call the agg method with a dictionary pairing the aggregating column with its aggregating function:

In [3]:
flights.groupby('AIRLINE')['ARR_DELAY'].agg('mean').head() #Average airline delay

AIRLINE
AA    5.542661
AS   -0.833333
B6    8.692593
DL    0.339691
EV    7.034580
Name: ARR_DELAY, dtype: float64

Alternatively, you may place the aggregating column in the indexing operator and then pass the aggregating function as a string to agg:

In [4]:
flights.groupby('AIRLINE').agg({'ARR_DELAY':'mean'}).head()

,ARR_DELAY
AIRLINE,
AA,5.542661
AS,-0.833333
B6,8.692593
DL,0.339691
EV,7.034580


The string names used in the previous step are a convenience pandas offers you to refer to a particular aggregation function. You can pass any aggregating function directly to the agg method such as the NumPy mean function. The output is the same as the previous step:

In [5]:
flights.groupby('AIRLINE')['ARR_DELAY'].agg(np.mean).head()

AIRLINE
AA    5.542661
AS   -0.833333
B6    8.692593
DL    0.339691
EV    7.034580
Name: ARR_DELAY, dtype: float64

It's possible to skip the agg method altogether in this case and use the mean method directly. This output is also the same as step 3:

In [5]:
flights.groupby('AIRLINE')['ARR_DELAY'].mean().head()

AIRLINE
AA    5.542661
AS   -0.833333
B6    8.692593
DL    0.339691
EV    7.034580
Name: ARR_DELAY, dtype: float64

## How it works...

The syntax for the groupby method is not as straightforward as other methods. Let's intercept the chain of methods in step 2 by storing the result of the groupby method as its own variable



In [6]:
grouped = flights.groupby('AIRLINE')
type(grouped)

pandas.core.groupby.generic.DataFrameGroupBy

A completely new intermediate object is first produced with its own distinct attributes and methods. No calculations take place at this stage. Pandas merely validates the grouping columns. This groupby object has an agg method to perform aggregations. One of the ways to use this method is to pass it a dictionary mapping the aggregating column to the aggregating function, as done in step 2.

There are several different flavors of syntax that produce a similar result, with step 3 showing an alternative. Instead of identifying the aggregating column in the dictionary, place it inside the indexing operator just as if you were selecting it as a column from a DataFrame. The function string name is then passed as a scalar to the agg method.

You may pass any aggregating function to the agg method. Pandas allows you to use the string names for simplicity but you may also explicitly call an aggregating function as done in step 4. NumPy provides many functions that aggregate values.

Step 5 shows one last syntax flavor. When you are only applying a single aggregating function as in this example, you can often call it directly as a method on the groupby object itself without agg. Not all aggregation functions have a method equivalent but many basic ones do. The following is a list of several aggregating functions that may be passed as a string to agg or chained directly as a method to the groupby object:

`min     max    mean    median    sum    count    std    var   
size    describe    nunique     idxmin     idxmax`

### There's more...

If you do not use an aggregating function with agg, pandas raises an exception. For instance, let's see what happens when we apply the square root function to each group:

In [7]:
flights.groupby('AIRLINE')['ARR_DELAY'].agg(np.sqrt)

C:\Users\Sumanth\anaconda3\envs\stablediffusion\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


ValueError: Must produce aggregated value

# Grouping and aggregating with multiple columns and functions
It is possible to do grouping and aggregating with multiple columns. The syntax is only slightly different than it is for grouping and aggregating with a single column. As usual with any kind of grouping operation, it helps to identify the three components: the grouping columns, aggregating columns, and aggregating functions.

### Getting ready

In this recipe, we showcase the flexibility of the groupby DataFrame method by answering the following queries:

Finding the number of cancelled flights for every airline per weekday

Finding the number and percentage of cancelled and diverted flights for every airline per weekday

For each origin and destination, finding the total number of flights, the number and percentage of cancelled flights, and the average and variance of the airtime

### How to do it...

Read in the flights dataset, and answer the first query by defining the grouping columns (AIRLINE, WEEKDAY), the aggregating column (CANCELLED), and the aggregating function (sum):

In [8]:
flights = pd.read_csv('data/flights.csv')
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


In [9]:
# The number of cancelled flights for every airline per day weekday
flights.groupby(['AIRLINE', 'WEEKDAY'])['CANCELLED'].agg('sum').head(7)

AIRLINE  WEEKDAY
AA       1          41
         2           9
         3          16
         4          20
         5          18
         6          21
         7          29
Name: CANCELLED, dtype: int64

Answer the second query by using a list for each pair of grouping and aggregating columns. Also, use a list for the aggregating functions:

In [11]:
#Find the number and percentage of cancelled and diverted flights for every airline per weekday
#First we groupby on airline and weekday then select columns cancelled and diverted. After that calculate sum and mean for each column
flights.groupby(['AIRLINE', 'WEEKDAY'])[['CANCELLED', 'DIVERTED']].agg(['sum', 'mean']).head(7)

CANCELLED           DIVERTED          
                      sum      mean      sum      mean
AIRLINE WEEKDAY                                       
AA      1              41  0.032106        6  0.004699
        2               9  0.007341        2  0.001631
        3              16  0.011949        2  0.001494
        4              20  0.015004        5  0.003751
        5              18  0.014151        1  0.000786
        6              21  0.018667        9  0.008000
        7              29  0.021837        1  0.000753

Answer the third query using a dictionary in the agg method to map specific aggregating columns to specific aggregating functions:

In [12]:
# For each origin to destination flight, find the total number of flights, 
# the number and percentage of cancelled flights and the average and variance of the airtime. 
flights.groupby(['ORG_AIR', 'DEST_AIR']).agg({'CANCELLED': ['sum', 'mean', 'size'], 
                                              'AIR_TIME':['mean', 'var']}).head()

CANCELLED              AIR_TIME           
                       sum mean size        mean        var
ORG_AIR DEST_AIR                                           
ATL     ABE              0  0.0   31   96.387097  45.778495
        ABQ              0  0.0   16  170.500000  87.866667
        ABY              0  0.0   19   28.578947   6.590643
        ACY              0  0.0    6   91.333333  11.466667
        AEX              0  0.0   40   78.725000  47.332692

### How it works...

To group by multiple columns as in step 1, we pass a list of the string names to the groupby method. Each unique combination of AIRLINE and WEEKDAY forms an independent group. Within each of these groups, the sum of the cancelled flights is found and then returned as a Series.

Step 2, again groups by both AIRLINE and WEEKDAY, but this time aggregates two columns. It applies each of the two aggregation functions, sum and mean, to each column resulting in four returned columns per group.

Step 3 goes even further, and uses a dictionary to map specific aggregating columns to different aggregating functions. Notice that the size aggregating function returns the total number of rows per group. This is different than the count aggregating function, which returns the number of non-missing values per group.

### There's more...

There are a few main flavors of syntax that you will encounter when performing an aggregation. The following four blocks of pseudocode summarize the main ways you can perform an aggregation with the groupby method:

Using agg with a dictionary is the most flexible and allows you to specify the aggregating function for each column:

In [14]:
#df.groupby(['grouping', 'columns']) \
#      .agg({'agg_cols1':['list', 'of', 'functions'], 
#            'agg_cols2':['other', 'functions']})

Using agg with a list of aggregating functions applies each of the functions to each of the aggregating columns:

In [15]:
#df.groupby(['grouping', 'columns'])['aggregating', 'columns'] \
#      .agg([aggregating, functions])

Directly using a method following the aggregating columns instead of agg, applies just that method to each aggregating column. This way does not allow for multiple aggregating functions:

In [ ]:
#df.groupby(['grouping', 'columns'])['aggregating', 'columns'] \
#      .aggregating_method()

If you do not specify the aggregating columns, then the aggregating method will be applied to all the non-grouping columns:

In [16]:
#df.groupby(['grouping', 'columns']).aggregating_method()

In the preceding four code blocks it is possible to substitute a string for any of the lists when grouping or aggregating by a single column.

# Removing the MultiIndex after grouping
Inevitably, when using groupby, you will likely create a MultiIndex in the columns or rows or both. DataFrames with MultiIndexes are more difficult to navigate and occasionally have confusing column names as well.

### Getting ready

In this recipe, we perform an aggregation with the groupby method to create a DataFrame with a MultiIndex for the rows and columns and then manipulate it so that the index is a single level and the column names are descriptive.

### How to do it...

`Read in the flights dataset; write a statement to find the total and average miles flown; and the maximum and minimum arrival delay for each airline for each weekday:`

In [19]:
flights = pd.read_csv('data/flights.csv')
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


In [20]:
airline_info = flights.groupby(['AIRLINE', 'WEEKDAY'])\
                      .agg({'DIST':['sum', 'mean'], 
                                    'ARR_DELAY':['min', 'max']}).astype(int)
airline_info.head()

DIST       ARR_DELAY     
                     sum  mean       min  max
AIRLINE WEEKDAY                              
AA      1        1455386  1139       -60  551
        2        1358256  1107       -52  725
        3        1496665  1117       -45  473
        4        1452394  1089       -46  349
        5        1427749  1122       -41  732

Both the rows and columns are labeled by a MultiIndex with two levels. Let's squash it down to just a single level. To address the columns, we use the MultiIndex method, get_level_values. Let's display the output of each level and then concatenate both levels before setting it as the new column values:

In [21]:
level0 = airline_info.columns.get_level_values(0)
level0

Index(['DIST', 'DIST', 'ARR_DELAY', 'ARR_DELAY'], dtype='object')

In [22]:
level1 = airline_info.columns.get_level_values(1)
level1

Index(['sum', 'mean', 'min', 'max'], dtype='object')

In [23]:
airline_info.columns = level0 + '_' + level1

In [24]:
airline_info.head(7)

DIST_sum  DIST_mean  ARR_DELAY_min  ARR_DELAY_max
AIRLINE WEEKDAY                                                   
AA      1         1455386       1139            -60            551
        2         1358256       1107            -52            725
        3         1496665       1117            -45            473
        4         1452394       1089            -46            349
        5         1427749       1122            -41            732
        6         1265340       1124            -50            858
        7         1461906       1100            -49            626

Return the row labels to a single level with reset_index:

In [25]:
airline_info.reset_index().head(7)

,AIRLINE,WEEKDAY,DIST_sum,DIST_mean,ARR_DELAY_min,ARR_DELAY_max
0,AA,1,1455386,1139,-60,551
1,AA,2,1358256,1107,-52,725
2,AA,3,1496665,1117,-45,473
3,AA,4,1452394,1089,-46,349
4,AA,5,1427749,1122,-41,732
5,AA,6,1265340,1124,-50,858
6,AA,7,1461906,1100,-49,626


### How it works...

When using the agg method to perform an aggregation on multiple columns, pandas creates an index object with two levels. The aggregating columns become the top level and the aggregating functions become the bottom level. Pandas displays MultiIndex levels differently than single-level columns. Except for the innermost levels, repeated index values do not get displayed on the screen. You can inspect the DataFrame from step 1 to verify this. For instance, the DIST column shows up only once but it refers to both of the first two columns.

**Note**

The innermost MultiIndex level is the one closest to the data. This would be the bottom-most column level and the right-most index level.

Step 2 defines new columns by first retrieving the underlying values of each of the levels with the MultiIndex method get_level_values. This method accepts an integer identifying the index level. They are numbered beginning with zero from the top/left. Indexes support vectorized operations, so we concatenate both levels together with a separating underscore. We assign these new values to the columns attribute.

In step 3, we make both index levels as columns with reset_index. We could have concatenated the levels together like we did in step 2, but it makes more sense to keep them as separate columns.

### There's more...

By default, at the end of a groupby operation, pandas puts all of the grouping columns in the index. The as_index parameter in the groupby method can be set to False to avoid this behavior. You can chain the reset_index method after grouping to get the same effect as done in step 3. Let's see an example of this by finding the average distance traveled per flight from each airline:

In [26]:
flights.groupby(['AIRLINE'], as_index=False)['DIST'].agg('mean').round(0)

,AIRLINE,DIST
0,AA,1114.0
1,AS,1066.0
2,B6,1772.0
3,DL,866.0
4,EV,460.0
5,F9,970.0
6,HA,2615.0
7,MQ,404.0
8,NK,1047.0
9,OO,511.0


In [27]:
flights.groupby(['AIRLINE'], as_index=False, sort=False)['DIST'].agg('mean')

,AIRLINE,DIST
0,WN,809.985626
1,UA,1230.918891
2,MQ,404.229041
3,AA,1114.347865
4,F9,969.593014
5,EV,460.237453
6,OO,511.239375
7,NK,1047.428100
8,US,1181.226625
9,AS,1065.884115


Take a look at the order of the airlines in the previous result. By default, pandas sorts the grouping columns. The sort parameter exists within the groupby method and is defaulted to True. You may set it to False to keep the order of the grouping columns the same as how they are encountered in the dataset. You also get a small performance improvement by not sorting your data.

# Customizing an aggregation function
Pandas provides a number of the most common aggregation functions for you to use with the groupby object. At some point, you will need to write your own customized user-defined functions that don't exist in pandas or NumPy.

### Getting ready

`In this recipe, we use the college dataset to calculate the mean and standard deviation of the undergraduate student population per state. We then use this information to find the maximum number of standard deviations from the mean that any single population value is per state.`

### How to do it...

Read in the college dataset, and find the mean and standard deviation of the undergraduate population by state:


In [28]:
college = pd.read_csv('data/college.csv')
college.head()

,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
0,Alabama A & M University,Normal,AL,1.0,0.0,0.0,0,424.0,420.0,0.0,...,0.0000,0.0059,0.0138,0.0656,1,0.7356,0.8284,0.1049,30300,33888
1,University of Alabama at Birmingham,Birmingham,AL,0.0,0.0,0.0,0,570.0,565.0,0.0,...,0.0368,0.0179,0.0100,0.2607,1,0.3460,0.5214,0.2422,39700,21941.5
2,Amridge University,Montgomery,AL,0.0,0.0,0.0,1,NaN,NaN,1.0,...,0.0000,0.0000,0.2715,0.4536,1,0.6801,0.7795,0.8540,40100,23370
3,University of Alabama in Huntsville,Huntsville,AL,0.0,0.0,0.0,0,595.0,590.0,0.0,...,0.0172,0.0332,0.0350,0.2146,1,0.3072,0.4596,0.2640,45500,24097
4,Alabama State University,Montgomery,AL,1.0,0.0,0.0,0,425.0,430.0,0.0,...,0.0098,0.0243,0.0137,0.0892,1,0.7347,0.7554,0.1270,26600,33118.5


In [29]:
college.groupby('STABBR')['UGDS'].agg(['mean', 'std']).round(0).head()

,mean,std
STABBR,,
AK,2493.0,4052.0
AL,2790.0,4658.0
AR,1644.0,3143.0
AS,1276.0,NaN
AZ,4130.0,14894.0


This output isn't quite what we desire. We are not looking for the mean and standard deviations of the entire group but the maximum number of standard deviations away from the mean for any one institution. In order to calculate this, we need to subtract the mean undergraduate population by state from each institution's undergraduate population and then divide by the standard deviation. This standardizes the undergraduate population for each group. We can then take the maximum of the absolute value of these scores to find the one that is farthest away from the mean. Pandas does not provide a function capable of doing this. Instead, we will need to create a custom function:

In [30]:
def max_deviation(s):
    std_score = (s - s.mean()) / s.std()
    return std_score.abs().max()

After defining the function, pass it directly to the agg method to complete the aggregation:

In [31]:
college.groupby('STABBR')['UGDS'].agg(max_deviation).round(1).head()

STABBR
AK    2.6
AL    5.8
AR    6.3
AS    NaN
AZ    9.9
Name: UGDS, dtype: float64

### How it works...

There does not exist a predefined pandas function to calculate the maximum number of standard deviations away from the mean. We were forced to construct a customized function in step 2. Notice that this custom function max_deviation accepts a single parameter, s. Looking ahead at step 3, you will notice that the function name is placed inside the agg method without directly being called. Nowhere is the parameter s explicitly passed to max_deviation. Instead, pandas implicitly passes the UGDS column as a Series to max_deviation.

The max_deviation function is called once for each group. As s is a Series, all normal Series methods are available. It subtracts the mean of that particular grouping from each of the values in the group before dividing by the standard deviation in a process called standardization.

**Note**

Standardization is a common statistical procedure to understand how greatly individual values vary from the mean. For a normal distribution, 99.7% of the data lies within three standard deviations of the mean.

As we are interested in absolute deviation from the mean, we take the absolute value from all the standardized scores and return the maximum. The agg method necessitates that a single scalar value must be returned from our custom function, or else an exception will be raised. Pandas defaults to using the sample standard deviation which is undefined for any groups with just a single value. For instance, the state abbreviation AS (American Samoa) has a missing value returned as it has only a single institution in the dataset.

### There's more...

It is possible to apply our customized function to multiple aggregating columns. We simply add more column names to the indexing operator. The max_deviation function only works with numeric columns:

In [33]:
college.groupby('STABBR')[['UGDS', 'SATVRMID', 'SATMTMID']].agg(max_deviation).round(1).head()

,UGDS,SATVRMID,SATMTMID
STABBR,,,
AK,2.6,NaN,NaN
AL,5.8,1.6,1.8
AR,6.3,2.2,2.3
AS,NaN,NaN,NaN
AZ,9.9,1.9,1.4


You can also use your customized aggregation function along with the prebuilt functions. The following does this and groups by state and religious affiliation:

In [34]:
college.groupby(['STABBR', 'RELAFFIL'])[['UGDS', 'SATVRMID', 'SATMTMID']]\
       .agg([max_deviation, 'mean', 'std']).round(1).head()

UGDS                      SATVRMID               \
                max_deviation    mean     std max_deviation   mean   std   
STABBR RELAFFIL                                                            
AK     0                  2.1  3508.9  4539.5           NaN    NaN   NaN   
       1                  1.1   123.3   132.9           NaN  555.0   NaN   
AL     0                  5.2  3248.8  5102.4           1.6  514.9  56.5   
       1                  2.4   979.7   870.8           1.5  498.0  53.0   
AR     0                  5.8  1793.7  3401.6           1.9  481.1  37.9   

                     SATMTMID               
                max_deviation   mean   std  
STABBR RELAFFIL                             
AK     0                  NaN    NaN   NaN  
       1                  NaN  503.0   NaN  
AL     0                  1.7  515.8  56.7  
       1                  1.4  485.6  61.4  
AR     0                  2.0  503.6  39.0

Notice that pandas uses the name of the function as the name for the returned column. You can change the column name directly with the rename method or you can modify the special function attribute __name__:

In [35]:
max_deviation.__name__

'max_deviation'

In [36]:
max_deviation.__name__ = 'Max Deviation'

In [37]:
college.groupby(['STABBR', 'RELAFFIL'])[['UGDS', 'SATVRMID', 'SATMTMID']]\
       .agg([max_deviation, 'mean', 'std']).round(1).head()

UGDS                      SATVRMID               \
                Max Deviation    mean     std Max Deviation   mean   std   
STABBR RELAFFIL                                                            
AK     0                  2.1  3508.9  4539.5           NaN    NaN   NaN   
       1                  1.1   123.3   132.9           NaN  555.0   NaN   
AL     0                  5.2  3248.8  5102.4           1.6  514.9  56.5   
       1                  2.4   979.7   870.8           1.5  498.0  53.0   
AR     0                  5.8  1793.7  3401.6           1.9  481.1  37.9   

                     SATMTMID               
                Max Deviation   mean   std  
STABBR RELAFFIL                             
AK     0                  NaN    NaN   NaN  
       1                  NaN  503.0   NaN  
AL     0                  1.7  515.8  56.7  
       1                  1.4  485.6  61.4  
AR     0                  2.0  503.6  39.0

# Customizing aggregating functions with kwargs and args
When writing your own user-defined customized aggregation function, pandas implicitly passes it each of the aggregating columns one at a time as a Series. Occasionally, you will need to pass more arguments to your function than just the Series itself. To do so, you need to be aware of Python's ability to pass an arbitrary number of arguments to functions. Let's take a look at the signature of the groupby object's agg method with help from the inspect module:


In [38]:
college = pd.read_csv('data/college.csv')
grouped = college.groupby(['STABBR', 'RELAFFIL'])

In [39]:
import inspect
inspect.signature(grouped.agg)

<Signature (func=None, *args, engine=None, engine_kwargs=None, **kwargs)>

The argument *args allow you to pass an arbitrary number of non-keyword arguments to your customized aggregation function. Similarly, **kwargs allows you to pass an arbitrary number of keyword arguments.

### Getting ready

In this recipe, we build a customized function for the college dataset that finds the percentage of schools by state and religious affiliation that have an undergraduate population between two values.

### How to do it...

Define a function that returns the percentage of schools with an undergraduate population between 1,000 and 3,000:

In [40]:
def pct_between_1_3k(s):
    return s.between(1000, 3000).mean()

Calculate this percentage grouping by state and religious affiliation:

In [41]:
college.groupby(['STABBR', 'RELAFFIL'])['UGDS'].agg(pct_between_1_3k).head(9)

STABBR  RELAFFIL
AK      0           0.142857
        1           0.000000
AL      0           0.236111
        1           0.333333
AR      0           0.279412
        1           0.111111
AS      0           1.000000
AZ      0           0.096774
        1           0.000000
Name: UGDS, dtype: float64

This function works fine but it doesn't give the user any flexibility to choose the lower and upper bound. Let's create a new function that allows the user to define these bounds:

In [42]:
def pct_between(s, low, high):
    return s.between(low, high).mean()

Pass this new function to the agg method along with lower and upper bounds:


In [43]:
college.groupby(['STABBR', 'RELAFFIL'])['UGDS'].agg(pct_between, 1000, 10000).head(9)

STABBR  RELAFFIL
AK      0           0.428571
        1           0.000000
AL      0           0.458333
        1           0.375000
AR      0           0.397059
        1           0.166667
AS      0           1.000000
AZ      0           0.233871
        1           0.111111
Name: UGDS, dtype: float64

### How it works...

Step 1 creates a function that doesn't accept any extra arguments. The upper and lower bounds must be hardcoded into the function itself, which isn't very flexible. Step 2 shows the results of this aggregation.

We create a more flexible function in step 3 that allows users to define both the lower and upper bounds dynamically. Step 4 is where the magic of *args and **kwargs come into play. In this particular example, we pass two non-keyword arguments, 1,000 and 10,000, to the agg method. Pandas passes these two arguments respectively to the low and high parameters of pct_between.

There are a few ways we could achieve the same result in step 4. We could have explicitly used the parameter names with the following command to produce the same result:

In [44]:
college.groupby(['STABBR', 'RELAFFIL'])['UGDS'].agg(pct_between, high=10000, low=1000).head(9)

STABBR  RELAFFIL
AK      0           0.428571
        1           0.000000
AL      0           0.458333
        1           0.375000
AR      0           0.397059
        1           0.166667
AS      0           1.000000
AZ      0           0.233871
        1           0.111111
Name: UGDS, dtype: float64

The order of the keyword arguments doesn't matter as long as they come after the function name. Further still, we can mix non-keyword and keyword arguments as long as the keyword arguments come last:

In [45]:
college.groupby(['STABBR', 'RELAFFIL'])['UGDS'].agg(pct_between, 1000, high=10000).head(9)

STABBR  RELAFFIL
AK      0           0.428571
        1           0.000000
AL      0           0.458333
        1           0.375000
AR      0           0.397059
        1           0.166667
AS      0           1.000000
AZ      0           0.233871
        1           0.111111
Name: UGDS, dtype: float64

For ease of understanding, it's probably best to include all the parameter names in the order that they are defined in the function signature.

**Note**

Technically, when agg is called, all the non-keyword arguments get collected into a tuple named args and all the keyword arguments get collected into a dictionary named kwargs.

### There's more...

Unfortunately, pandas does not have a direct way to use these additional arguments when using multiple aggregation functions together. For example, if you wish to aggregate using the pct_between and mean functions, you will get the following exception:



In [46]:
college.groupby(['STABBR', 'RELAFFIL'])['UGDS'].agg(['mean', pct_between], low=100, high=1000)

TypeError: GroupBy.mean() got an unexpected keyword argument 'low'

Pandas is incapable of understanding that the extra arguments need to be passed to pct_between. In order to use our custom function with other built-in functions and even other custom functions, we can define a special type of nested function called a closure. We can use a generic closure to build all of our customized functions:



In [47]:
def make_agg_func(func, name, *args, **kwargs):
    def wrapper(x):
        return func(x, *args, **kwargs)
    wrapper.__name__ = name
    return wrapper

my_agg1 = make_agg_func(pct_between, 'pct_1_3k', low=1000, high=3000)
my_agg2 = make_agg_func(pct_between, 'pct_10_30k', 10000, 30000)

In [48]:
college.groupby(['STABBR', 'RELAFFIL'])['UGDS'].agg(['mean', my_agg1, my_agg2]).head()

mean  pct_1_3k  pct_10_30k
STABBR RELAFFIL                                   
AK     0         3508.857143  0.142857    0.142857
       1          123.333333  0.000000    0.000000
AL     0         3248.774648  0.236111    0.083333
       1          979.722222  0.333333    0.000000
AR     0         1793.691176  0.279412    0.014706

The make_agg_func function acts as a factory to create customized aggregation functions. It accepts the customized aggregation function that you already built (pct_between in this case), a name argument, and an arbitrary number of extra arguments. It returns a function with the extra arguments already set. For instance, my_agg1 is a specific customized aggregating function that finds the percentage of schools with an undergraduate population between one and three thousand. The extra arguments (*args and **kwargs) specify an exact set of parameters for your customized function (pct_between in this case). The name parameter is very important and must be unique each time make_agg_func is called. It will eventually be used to rename the aggregated column.

**Note**

A closure is a function that contains a function inside of it (a nested function) and returns this nested function. This nested function must refer to variables in the scope of the outer function in order to be a closure. In this example, make_agg_func is the outer function and returns the nested function wrapper, which accesses the variables func, args, and kwargs from the outer function.



# Examining a groupby object
The immediate result from using the groupby method on a DataFrame will be a groupby object. Usually, we continue operating on this object to do aggregations or transformations without ever saving it to a variable. One of the primary purposes of examining this groupby object is to inspect individual groups.

### Getting ready

In this recipe, we examine the groupby object itself by directly calling methods on it as well as iterating through each of its groups.

### How to do it...

Let's get started by grouping the state and religious affiliation columns from the college dataset, saving the result to a variable and confirming its type:

In [49]:
college = pd.read_csv('data/college.csv')
grouped = college.groupby(['STABBR', 'RELAFFIL'])
type(grouped)

pandas.core.groupby.generic.DataFrameGroupBy

Use the dir function to discover all its available functionality:

In [50]:
print([attr for attr in dir(grouped) if not attr.startswith('_')])

['CITY', 'CURROPER', 'DISTANCEONLY', 'GRAD_DEBT_MDN_SUPP', 'HBCU', 'INSTNM', 'MD_EARN_WNE_P10', 'MENONLY', 'PCTFLOAN', 'PCTPELL', 'PPTUG_EF', 'RELAFFIL', 'SATMTMID', 'SATVRMID', 'STABBR', 'UG25ABV', 'UGDS', 'UGDS_2MOR', 'UGDS_AIAN', 'UGDS_ASIAN', 'UGDS_BLACK', 'UGDS_HISP', 'UGDS_NHPI', 'UGDS_NRA', 'UGDS_UNKN', 'UGDS_WHITE', 'WOMENONLY', 'agg', 'aggregate', 'all', 'any', 'apply', 'bfill', 'boxplot', 'corr', 'corrwith', 'count', 'cov', 'cumcount', 'cummax', 'cummin', 'cumprod', 'cumsum', 'describe', 'diff', 'dtypes', 'ewm', 'expanding', 'ffill', 'fillna', 'filter', 'first', 'get_group', 'groups', 'head', 'hist', 'idxmax', 'idxmin', 'indices', 'last', 'max', 'mean', 'median', 'min', 'ndim', 'ngroup', 'ngroups', 'nth', 'nunique', 'ohlc', 'pct_change', 'pipe', 'plot', 'prod', 'quantile', 'rank', 'resample', 'rolling', 'sample', 'sem', 'shift', 'size', 'skew', 'std', 'sum', 'tail', 'take', 'transform', 'value_counts', 'var']


Find the number of groups with the ngroups attribute:

In [51]:
grouped.ngroups

112

To find the uniquely identifying labels for each group, look in the groups attribute, which contains a dictionary of each unique group mapped to all the corresponding index labels of that group:

In [52]:
groups = list(grouped.groups.keys())
groups[:6]

[('AK', 0), ('AK', 1), ('AL', 0), ('AL', 1), ('AR', 0), ('AR', 1)]

Retrieve a single group with the get_group method by passing it a tuple of an exact group label. For example, to get all the religiously affiliated schools in the state of Florida, do the following:

In [53]:
grouped.get_group(('FL', 1)).head()

,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
712,The Baptist College of Florida,Graceville,FL,0.0,0.0,0.0,1,545.0,465.0,0.0,...,0.0308,0.0000,0.0507,0.2291,1,0.5878,0.5602,0.3531,30800,20052
713,Barry University,Miami,FL,0.0,0.0,0.0,1,470.0,462.0,0.0,...,0.0164,0.0741,0.0841,0.1518,1,0.5045,0.6733,0.4361,44100,28250
714,Gooding Institute of Nurse Anesthesia,Panama City,FL,0.0,0.0,0.0,1,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,PrivacySuppressed
715,Bethune-Cookman University,Daytona Beach,FL,1.0,0.0,0.0,1,405.0,395.0,0.0,...,0.0198,0.0205,0.0190,0.0523,1,0.7758,0.8867,0.0647,29400,36250
724,Johnson University Florida,Kissimmee,FL,0.0,0.0,0.0,1,480.0,470.0,0.0,...,0.0045,0.0045,0.0136,0.1636,1,0.6689,0.7384,0.2185,26300,20199


You may want to take a peek at each individual group. This is possible because groupby objects are iterable:

In [54]:
from IPython.display import display

In [55]:
i = 0
for name, group in grouped:
    print(name)
    display(group.head(2))
    i += 1
    if i == 5:
        break

('AK', 0)


,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
60,University of Alaska Anchorage,Anchorage,AK,0.0,0.0,0.0,0,NaN,NaN,0.0,...,0.0980,0.0181,0.0457,0.4539,1,0.2385,0.2647,0.4386,42500,19449.5
62,University of Alaska Fairbanks,Fairbanks,AK,0.0,0.0,0.0,0,NaN,NaN,0.0,...,0.0401,0.0110,0.3060,0.3887,1,0.2263,0.2550,0.4519,36200,19355


('AK', 1)


,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
61,Alaska Bible College,Palmer,AK,0.0,0.0,0.0,1,NaN,NaN,0.0,...,0.0370,0.0,0.0000,0.1481,1,0.3571,0.2857,0.4286,NaN,PrivacySuppressed
64,Alaska Pacific University,Anchorage,AK,0.0,0.0,0.0,1,555.0,503.0,0.0,...,0.0945,0.0,0.0873,0.3745,1,0.3152,0.5297,0.4910,47000,23250


('AL', 0)


,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
0,Alabama A & M University,Normal,AL,1.0,0.0,0.0,0,424.0,420.0,0.0,...,0.0000,0.0059,0.0138,0.0656,1,0.7356,0.8284,0.1049,30300,33888
1,University of Alabama at Birmingham,Birmingham,AL,0.0,0.0,0.0,0,570.0,565.0,0.0,...,0.0368,0.0179,0.0100,0.2607,1,0.3460,0.5214,0.2422,39700,21941.5


('AL', 1)


,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
2,Amridge University,Montgomery,AL,0.0,0.0,0.0,1,NaN,NaN,1.0,...,0.0000,0.0,0.2715,0.4536,1,0.6801,0.7795,0.8540,40100,23370
10,Birmingham Southern College,Birmingham,AL,0.0,0.0,0.0,1,560.0,560.0,0.0,...,0.0051,0.0,0.0051,0.0017,1,0.1920,0.4809,0.0152,44200,27000


('AR', 0)


,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
128,University of Arkansas at Little Rock,Little Rock,AR,0.0,0.0,0.0,0,470.0,510.0,0.0,...,0.0755,0.0283,0.0003,0.4126,1,0.3941,0.4775,0.4062,33900,21736
129,University of Arkansas for Medical Sciences,Little Rock,AR,0.0,0.0,0.0,0,NaN,NaN,0.0,...,0.0281,0.0070,0.0169,0.2433,1,0.3944,0.6144,0.5133,61400,12500


You can also call the head method on your groupby object to get the first rows of each group together in a single DataFrame.

In [56]:
grouped.head(2).head(6)

,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
0,Alabama A & M University,Normal,AL,1.0,0.0,0.0,0,424.0,420.0,0.0,...,0.0000,0.0059,0.0138,0.0656,1,0.7356,0.8284,0.1049,30300,33888
1,University of Alabama at Birmingham,Birmingham,AL,0.0,0.0,0.0,0,570.0,565.0,0.0,...,0.0368,0.0179,0.0100,0.2607,1,0.3460,0.5214,0.2422,39700,21941.5
2,Amridge University,Montgomery,AL,0.0,0.0,0.0,1,NaN,NaN,1.0,...,0.0000,0.0000,0.2715,0.4536,1,0.6801,0.7795,0.8540,40100,23370
10,Birmingham Southern College,Birmingham,AL,0.0,0.0,0.0,1,560.0,560.0,0.0,...,0.0051,0.0000,0.0051,0.0017,1,0.1920,0.4809,0.0152,44200,27000
43,Prince Institute-Southeast,Elmhurst,IL,0.0,0.0,0.0,0,NaN,NaN,0.0,...,0.0000,0.0000,0.0000,0.0000,1,0.7857,0.9375,0.6569,PrivacySuppressed,20992
60,University of Alaska Anchorage,Anchorage,AK,0.0,0.0,0.0,0,NaN,NaN,0.0,...,0.0980,0.0181,0.0457,0.4539,1,0.2385,0.2647,0.4386,42500,19449.5


### How it works...

Step 1 formally creates our groupby object. It is useful to display all the public attributes and methods to reveal all the possible functionality as was done in step 2. Each group is uniquely identified by a tuple containing a unique combination of the values in the grouping columns. Pandas allows you to select a specific group as a DataFrame with the get_group method shown in step 5.

It is rare that you will need to iterate through your groups and in general, you should avoid doing so if necessary, as it can be quite slow. Occasionally, you will have no other choice. When iterating through a groupby object, you are given a tuple containing the group name and the DataFrame without the grouping columns. This tuple is unpacked into the variables name and group in the for-loop in step 6.

One interesting thing you can do while iterating through your groups is to display a few of the rows from each group directly in the notebook. To do this, you can either use the print function or the display function from the IPython.display module. Using the print function results in DataFrames that are in plain text without any nice HTML formatting. Using the display function will produce DataFrames in their normal easy-to-read format. 

### There's more...

There are several useful methods that were not explored from the list in step 2. Take for instance the nth method, which, when given a list of integers, selects those specific rows from each group. For example, the following operation selects the first and last rows from each group:

In [57]:
grouped.nth([1, -1]).head(8)

,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
1,University of Alabama at Birmingham,Birmingham,AL,0.0,0.0,0.0,0,570.0,565.0,0.0,...,0.0368,0.0179,0.0100,0.2607,1,0.3460,0.5214,0.2422,39700,21941.5
10,Birmingham Southern College,Birmingham,AL,0.0,0.0,0.0,1,560.0,560.0,0.0,...,0.0051,0.0000,0.0051,0.0017,1,0.1920,0.4809,0.0152,44200,27000
62,University of Alaska Fairbanks,Fairbanks,AK,0.0,0.0,0.0,0,NaN,NaN,0.0,...,0.0401,0.0110,0.3060,0.3887,1,0.2263,0.2550,0.4519,36200,19355
64,Alaska Pacific University,Anchorage,AK,0.0,0.0,0.0,1,555.0,503.0,0.0,...,0.0945,0.0000,0.0873,0.3745,1,0.3152,0.5297,0.4910,47000,23250
70,Empire Beauty School-Paradise Valley,Phoenix,AZ,0.0,0.0,0.0,1,NaN,NaN,0.0,...,0.0400,0.0000,0.0000,0.1600,0,0.6349,0.5873,0.4651,17800,9588
71,Empire Beauty School-Tucson,Tucson,AZ,0.0,0.0,0.0,0,NaN,NaN,0.0,...,0.0000,0.0000,0.0079,0.2222,1,0.7962,0.6615,0.4229,18200,9833
129,University of Arkansas for Medical Sciences,Little Rock,AR,0.0,0.0,0.0,0,NaN,NaN,0.0,...,0.0281,0.0070,0.0169,0.2433,1,0.3944,0.6144,0.5133,61400,12500
134,Lyon College,Batesville,AR,0.0,0.0,0.0,1,505.0,528.0,0.0,...,0.0000,0.0333,0.0638,0.0101,1,0.4578,0.6740,0.0524,38600,25000


# Filtering for states with minimum
In Chapter 4, Selecting Subsets of Data, we marked every row as True or False before filtering out the False rows. In a similar fashion, it is possible to mark entire groups of data as either True or False before filtering out the False groups. To do this, we first form groups with the groupby method and then apply the filter method. The filter method accepts a function that must return either True or False to indicate whether a group is kept or not.

**Note**

This filter method applied after a call to the groupby method is completely different than the DataFrame filter method covered in the Selecting columns with methods recipe from Chapter 2, Essential DataFrame Operations.

### Getting ready

In this recipe, we use the college dataset to find all the states that have more non-white undergraduate students than white. As this is a dataset from the US, whites form the majority and therefore, we are looking for states with a minority majority.

### How to do it...

Read in the college dataset, group by state, and display the total number of groups. This should equal the number of unique states retrieved from the nunique Series method:

In [58]:
college = pd.read_csv('data/college.csv', index_col='INSTNM')
grouped = college.groupby('STABBR')
grouped.ngroups

59

In [59]:
%timeit college['STABBR'].nunique()

301 µs ± 4.05 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


The grouped variable has a filter method, which accepts a custom function that determines whether a group is kept or not. The custom function gets implicitly passed a DataFrame of the current group and is required to return a boolean. Let's define a function that calculates the total percentage of minority students and returns True if this percentage is greater than a user-defined threshold:

In [60]:
def check_minority(df, threshold):
    minority_pct = 1 - df['UGDS_WHITE']
    total_minority = (df['UGDS'] * minority_pct).sum()
    total_ugds = df['UGDS'].sum()
    total_minority_pct = total_minority / total_ugds
    return total_minority_pct > threshold

Use the filter method passed with the check_minority function and a threshold of 50% to find all states that have a minority majority:

In [61]:
college_filtered = grouped.filter(check_minority, threshold=.5)
college_filtered.head()

,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,UGDS,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
INSTNM,,,,,,,,,,,,,,,,,,,,,
Everest College-Phoenix,Phoenix,AZ,0.0,0.0,0.0,1,NaN,NaN,0.0,4102.0,...,0.0373,0.0,0.1026,0.4749,0,0.8291,0.7151,0.6700,28600,9500
Collins College,Phoenix,AZ,0.0,0.0,0.0,0,NaN,NaN,0.0,83.0,...,0.0241,0.0,0.3855,0.3373,0,0.7205,0.8228,0.4764,25700,47000
Empire Beauty School-Paradise Valley,Phoenix,AZ,0.0,0.0,0.0,1,NaN,NaN,0.0,25.0,...,0.0400,0.0,0.0000,0.1600,0,0.6349,0.5873,0.4651,17800,9588
Empire Beauty School-Tucson,Tucson,AZ,0.0,0.0,0.0,0,NaN,NaN,0.0,126.0,...,0.0000,0.0,0.0079,0.2222,1,0.7962,0.6615,0.4229,18200,9833
Thunderbird School of Global Management,Glendale,AZ,0.0,0.0,0.0,0,NaN,NaN,0.0,1.0,...,0.0000,0.0,0.0000,1.0000,0,0.0000,0.0000,0.0000,118900,PrivacySuppressed


Just looking at the output may not be indicative of what actually happened. The DataFrame starts with state Arizona (AZ) and not Alaska (AK) so we can visually confirm that something changed. Let's compare the shape of this filtered DataFrame with the original. Looking at the results, about 60% of the rows have been filtered, and only 20 states remain that have a minority majority:

In [62]:
college.shape

(7535, 26)

In [63]:
college_filtered.shape

(3028, 26)

In [64]:
college_filtered['STABBR'].nunique()

20

### How it works...

This recipe takes a look at the total population of all the institutions on a state-by-state basis. The goal is to keep all the rows from the states, as a whole, that have a minority majority. This requires us to group our data by state, which is done in step 1. We find that there are 59 independent groups.

The filter groupby method either keeps all the rows in a group or filters them out. It does not change the number of columns. The filter groupby method performs this gatekeeping through a user-defined function, for example, check_minority in this recipe. A very important aspect to filter is that it passes the entire DataFrame for that particular group to the user-defined function and returns a single boolean for each group.

Inside of the check_minority function, the percentage and the total number of non-white students for each institution are first calculated and then the total number of all students is found. Finally, the percentage of non-white students for the entire state is checked against the given threshold, which produces a boolean.

The final result is a DataFrame with the same columns as the original but with the rows from the states that don't meet the threshold filtered out. As it is possible that the head of the filtered DataFrame is the same as the original, you need to do some inspection to ensure that the operation completed successfully. We verify this by checking the number of rows and number of unique states.

### There's more...

Our function, check_minority, is flexible and accepts a parameter to lower or raise the percentage of minority threshold. Let's check the shape and number of unique states for a couple of other thresholds:

In [65]:
college_filtered_20 = grouped.filter(check_minority, threshold=.2)
college_filtered_20.shape

(7461, 26)

In [66]:
college_filtered_20['STABBR'].nunique()

57

In [67]:
college_filtered_70 = grouped.filter(check_minority, threshold=.7)
college_filtered_70.shape

(957, 26)

In [68]:
college_filtered_70['STABBR'].nunique()

10

In [69]:
college_filtered_95 = grouped.filter(check_minority, threshold=.95)
college_filtered_95.shape

(156, 26)

# Transforming through a weight-loss 
One method to increase motivation to lose weight is to make a bet with someone else. The scenario in this recipe will track weight loss from two individuals over the course of a four-month period and determine a winner.

### Getting ready

In this recipe, we use simulated data from two individuals to track the percentage of weight loss over the course of four months. At the end of each month, a winner will be declared based on the individual who lost the highest percentage of body weight for that month. To track weight loss, we group our data by month and person, then call the transform method to find the percentage weight loss at each week from the start of the month.

### How to do it...

Read in the raw weight_loss dataset, and examine the first month of data from the two people, Amy and Bob. There are a total of four weigh-ins per month:

In [70]:
weight_loss = pd.read_csv('data/weight_loss.csv')
weight_loss.query('Month == "Jan"')

,Name,Month,Week,Weight
0,Bob,Jan,Week 1,291
1,Amy,Jan,Week 1,197
2,Bob,Jan,Week 2,288
3,Amy,Jan,Week 2,189
4,Bob,Jan,Week 3,283
5,Amy,Jan,Week 3,189
6,Bob,Jan,Week 4,283
7,Amy,Jan,Week 4,190


To determine the winner for each month, we only need to compare weight loss from the first week to the last week of each month. But, if we wanted to have weekly updates, we can also calculate weight loss from the current week to the first week of each month.  Let's create a function that is capable of providing weekly updates:

In [71]:
def find_perc_loss(s):
    return (s - s.iloc[0]) / s.iloc[0]

In [72]:
bob_jan = weight_loss.query('Name=="Bob" and Month=="Jan"')
find_perc_loss(bob_jan['Weight'])

0    0.000000
2   -0.010309
4   -0.027491
6   -0.027491
Name: Weight, dtype: float64

After the first week, Bob lost 1% of his body weight. He continued losing weight during the second week but made no progress during the last week.  We can apply this function to every single combination of person and week to get the weight loss per week in relation to the first week of the month. To do this, we need to group our data by Name and Month , and then use the transform method to apply this custom function:

In [73]:
per_weight_loss = weight_loss.groupby(['Name', 'Month'])['Weight'].transform(find_perc_loss)
per_weight_loss.head(8)

0    0.000000
1    0.000000
2   -0.010309
3   -0.040609
4   -0.027491
5   -0.040609
6   -0.027491
7   -0.035533
Name: Weight, dtype: float64

The transform method must return an object with the same number of rows as the calling DataFrame. Let's append this result to our original DataFrame as a new column. To help shorten the output, we will select Bob's first two months of data:

In [74]:
weight_loss['Perc Weight Loss'] = per_weight_loss.round(3)
weight_loss.query('Name=="Bob" and Month in ["Jan", "Feb"]')

,Name,Month,Week,Weight,Perc Weight Loss
0,Bob,Jan,Week 1,291,0.000
2,Bob,Jan,Week 2,288,-0.010
4,Bob,Jan,Week 3,283,-0.027
6,Bob,Jan,Week 4,283,-0.027
8,Bob,Feb,Week 1,283,0.000
10,Bob,Feb,Week 2,275,-0.028
12,Bob,Feb,Week 3,268,-0.053
14,Bob,Feb,Week 4,268,-0.053


Notice that the percentage weight loss resets after the new month. With this new column, we can manually determine a winner but let's see if we can find a way to do this automatically. As the only week that matters is the last week, let's select week 4:

In [75]:
week4 = weight_loss.query('Week == "Week 4"')
week4

,Name,Month,Week,Weight,Perc Weight Loss
6,Bob,Jan,Week 4,283,-0.027
7,Amy,Jan,Week 4,190,-0.036
14,Bob,Feb,Week 4,268,-0.053
15,Amy,Feb,Week 4,173,-0.089
22,Bob,Mar,Week 4,261,-0.026
23,Amy,Mar,Week 4,170,-0.017
30,Bob,Apr,Week 4,250,-0.042
31,Amy,Apr,Week 4,161,-0.053


This narrows down the weeks but still doesn't automatically find out the winner of each month. Let's reshape this data with the pivot method so that Bob's and Amy's percent weight loss is side-by-side for each month:

In [76]:
winner = week4.pivot(index='Month', columns='Name', values='Perc Weight Loss')
winner

Name,Amy,Bob
Month,,
Apr,-0.053,-0.042
Feb,-0.089,-0.053
Jan,-0.036,-0.027
Mar,-0.017,-0.026


This output makes it clearer who has won each month, but we can still go a couple steps farther. NumPy has a vectorized if-then-else function called where, which can map a Series or array of booleans to other values. Let's create a column for the name of the winner and highlight the winning percentage for each month:

In [77]:
winner['Winner'] = np.where(winner['Amy'] < winner['Bob'], 'Amy', 'Bob')
winner.style.highlight_min(axis=1)

TypeError: '<=' not supported between instances of 'float' and 'str'

Use the value_counts method to return the final score as the number of months won:

In [78]:
winner.Winner.value_counts()

Winner
Amy    3
Bob    1
Name: count, dtype: int64

### How it works...

Throughout this recipe, the query method is used to filter data instead of boolean indexing. Refer to the Improving readability of Boolean indexing with the query method recipe from Chapter 5, Boolean Indexing, for more information.

Our goal is to find the percentage weight loss for each month for each person. One way to accomplish this task is to calculate each week's weight loss relative to the start of each month. This specific task is perfectly suited to the transformgroupby method. The transform method accepts a function as its one required parameter. This function gets implicitly passed each non-grouping column (or only the columns specified in the indexing operator as was done in this recipe with Weight). It must return a sequence of values the same length as the passed group or else an exception will be raised. In essence, all values from the original DataFrame are transforming. No aggregation or filtration takes place.

Step 2 creates a function that subtracts the first value of the passed Series from all of its values and then divides this result by the first value. This calculates the percent loss (or gain) relative to the first value. In step 3 we test this function on one person during one month.

In step 4, we use this function in the same manner over every combination of person and week. In some literal sense, we are transforming the Weight column into the percentage of weight lost for the current week. The first month of data is outputted for each person. Pandas returns the new data as a Series. This Series isn't all that useful by itself and makes more sense appended to the original DataFrame as a new column. We complete this operation in step 5.

To determine the winner, only week 4 of each month is necessary. We could stop here and manually determine the winner but pandas supplies us functionality to automate this. The pivot function in step 7 reshapes our dataset by pivoting the unique values of one column into new column names. The index parameter is used for the column that you do not want to pivot. The column passed to the values parameter gets tiled over each unique combination of the columns in the index and columns parameters.

**Note**

The pivot method only works if there is just a single occurrence of each unique combination of the columns in the index and columns parameters. If there is more than one unique combination, an exception will be raised. You can use the pivot_table method in that situation which allows you to aggregate multiple values together.

After pivoting, we utilize the highly effective and fast NumPy where function, whose first argument is a condition that produces a Series of booleans. True values get mapped to Amy and False values get mapped to Bob. We highlight the winner of each month and tally the final score with the value_counts method.

### There's more...

Take a look at the DataFrame output from step 7. Did you notice that the months are in alphabetical and not chronological order? Pandas unfortunately, in this case at least, orders the months for us alphabetically. We can solve this issue by changing the data type of Month to a categorical variable. Categorical variables map all the values of each column to an integer. We can choose this mapping to be the normal chronological order for the months. Pandas uses this underlying integer mapping during the pivot method to order the months chronologically:

In [79]:
week4a = week4.copy()
month_chron = week4a['Month'].unique() # or month.drop_duplicates
month_chron

array(['Jan', 'Feb', 'Mar', 'Apr'], dtype=object)

In [80]:
week4a['Month'] = pd.Categorical(week4a['Month'], categories=month_chron)
week4a.pivot(index='Month', columns='Name', values='Perc Weight Loss')

Name,Amy,Bob
Month,,
Jan,-0.036,-0.027
Feb,-0.089,-0.053
Mar,-0.017,-0.026
Apr,-0.053,-0.042


To convert the Month column, use the Categorical constructor. Pass it the original column as a Series and a unique sequence of all the categories in the desired order to the categories parameter. As the Month column is already in chronological order, we can simply use the unique method, which preserves order to get the array that we desire. In general, to sort columns of object data type by something other than alphabetical, convert them to categorical.

# Calculating weighted mean SAT scores per state with apply
The groupby object has four methods that accept a function (or functions) to perform a calculation on each group. These four methods are agg, filter, transform, and apply. Each of the first three of these methods has a very specific output that the function must return. agg must return a scalar value, filter must return a boolean, and transform must return a Series with the same length as the passed group. The apply method, however, may return a scalar value, a Series, or even a DataFrame of any shape, therefore making it very flexible. It is also called only once per group, which contrasts with transform and agg that get called once for each non-grouping column. The apply method's ability to return a single object when operating on multiple columns at the same time makes the calculation in this recipe possible.

### Getting ready

`In this recipe, we calculate the weighted average of both the math and verbal SAT scores per state from the college dataset. We weight the scores by the population of undergraduate students per school.`

### How to do it...

Read in the college dataset, and drop any rows that have missing values in either the UGDS, SATMTMID, or SATVRMID columns. We must have non-missing values for each of these three columns:

In [81]:
college = pd.read_csv('data/college.csv')
college2 = college.dropna(subset=['UGDS', 'SATMTMID', 'SATVRMID'])
college.shape

(7535, 27)

In [82]:
college2.shape

(1184, 27)

The vast majority of institutions do not have data for our three required columns, but this is still more than enough data to continue. Next, create a user-defined function to calculate the weighted average of just the SAT math scores:


In [83]:
def weighted_math_average(df):
    weighted_math = df['UGDS'] * df['SATMTMID']
    return int(weighted_math.sum() / df['UGDS'].sum())

Group by state and pass this function to the apply method:

In [84]:
college2.groupby('STABBR').apply(weighted_math_average).head()

STABBR
AK    503
AL    536
AR    529
AZ    569
CA    564
dtype: int64

We successfully returned a scalar value for each group. Let's take a small detour and see what the outcome would have been by passing the same function to the agg method:

In [85]:
college2.groupby('STABBR').agg(weighted_math_average).head()

KeyError: 'UGDS'

The weighted_math_average function gets applied to each non-aggregating column in the DataFrame. If you try and limit the columns to just SATMTMID, you will get an error as you won't have access to UGDS. So, the best way to complete operations that act on multiple columns is with apply:

In [86]:
college2.groupby('STABBR')['SATMTMID'].agg(weighted_math_average)

KeyError: 'UGDS'

A nice feature of apply is that you can create multiple new columns by returning a Series. The index of this returned Series will be the new column names. Let's modify our function to calculate the weighted and arithmetic average for both SAT scores along with the count of the number of institutions from each group. We return these five values in a Series:

In [87]:
from collections import OrderedDict
def weighted_average(df):
    data = OrderedDict()
    weighted_math = df['UGDS'] * df['SATMTMID']
    weighted_verbal = df['UGDS'] * df['SATVRMID']
    
    data['weighted_math_avg'] = weighted_math.sum() / df['UGDS'].sum()
    data['weighted_verbal_avg'] = weighted_verbal.sum() / df['UGDS'].sum()
    data['math_avg'] = df['SATMTMID'].mean()
    data['verbal_avg'] = df['SATVRMID'].mean()
    data['count'] = len(df)
    return pd.Series(data, dtype='int')

college2.groupby('STABBR').apply(weighted_average).head(10)

ValueError: Trying to coerce float values to integers

### How it works...

In order for this recipe to complete properly, we need to first filter for institutions that do not have missing values for UGDS, SATMTMID, and SATVRMID. By default, the dropna method drops rows that have one or more missing values. We must use the subset parameter to limit the columns it looks at for missing values.

In step 2, we define a function that calculates the weighted average for just the SATMTMID column. The weighted average differs from an arithmetic mean in that each value is multiplied by some weight. This quantity is then summed and divided by the sum of the weights. In this case, our weight is the undergraduate student population.

In step 3, we pass this function to the apply method. Our function weighted_math_average gets passed a DataFrame of all the original columns for each group. It returns a single scalar value, the weighted average of SATMTMID. At this point, you might think that this calculation is possible using the agg method. Directly replacing apply with agg does not work as agg returns a value for each of its aggregating columns.

**Note**

It actually is possible to use agg indirectly by precomputing the multiplication of UGDS and SATMTMID.

Step 6 really shows the versatility of apply. We build a new function that calculates the weighted and arithmetic average of both SAT columns as well as the number of rows for each group. In order for apply to create multiple columns, you must return a Series. The index values are used as column names in the resulting DataFrame. You can return as many values as you want with this method.

Notice that the OrderedDict class was imported from the collections module, which is part of the standard library. This ordered dictionary is used to store the data. A normal Python dictionary could not have been used to store the data since it does not preserve insertion order.

**Note**

The constructor, pd.Series, does have an index parameter that you can use to specify order but using an OrderedDict is cleaner.

### There's more...

In this recipe, we returned a single row as a Series for each group. It's possible to return any number of rows and columns for each group by returning a DataFrame. In addition to finding just the arithmetic and weighted means, let's also find the geometric and harmonic means of both SAT columns and return the results as a DataFrame with rows as the name of the type of mean and columns as the SAT type. To ease the burden on us, we use the NumPy function average to compute the weighted average and the SciPy functions gmean and hmean for geometric and harmonic means:



In [88]:
from scipy.stats import gmean, hmean
def calculate_means(df):
    df_means = pd.DataFrame(index=['Arithmetic', 'Weighted', 'Geometric', 'Harmonic'])
    cols = ['SATMTMID', 'SATVRMID']
    for col in cols:
        arithmetic = df[col].mean()
        weighted = np.average(df[col], weights=df['UGDS'])
        geometric = gmean(df[col])
        harmonic = hmean(df[col])
        df_means[col] = [arithmetic, weighted, geometric, harmonic]
        
    df_means['count'] = len(df)
    return df_means.astype(int)

college2.groupby('STABBR').filter(lambda x: len(x) != 1).groupby('STABBR').apply(calculate_means).head(10)

SATMTMID  SATVRMID  count
STABBR                                      
AL     Arithmetic       504       508     21
       Weighted         536       533     21
       Geometric        500       505     21
       Harmonic         497       502     21
AR     Arithmetic       515       491     16
       Weighted         529       504     16
       Geometric        514       489     16
       Harmonic         513       487     16
AZ     Arithmetic       536       538      6
       Weighted         569       557      6

# Grouping by continuous variables

When grouping in pandas, you typically use columns with discrete repeating values. If there are no repeated values, then grouping would be pointless as there would only be one row per group. Continuous numeric columns typically have few repeated values and are generally not used to form groups. However, if we can transform columns with continuous values into a discrete column by placing each value into a bin, rounding them, or using some other mapping, then grouping with them makes sense.

### Getting ready

In this recipe, we explore the flights dataset to discover the distribution of airlines for different travel distances. This allows us, for example, to find the airline that makes the most flights between 500 and 1,000 miles. To accomplish this, we use the pandas cut function to discretize the distance of each flight flown.

### How to do it...

Read in the flights dataset, and output the first five rows:

In [89]:
flights = pd.read_csv('data/flights.csv')
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


https://www.wunderground.com/history/airport/KDFW/2015/2/27/DailyHistory.html?req_city=Dallas-Fort+Worth+International&req_state=TX&req_statename=Texas&reqdb.zip=75261&reqdb.magic=5&reqdb.wmo=99999&MR=1

In [90]:
flights.DIST.hasnans

False

In [91]:
flights.dropna(subset=['DIST']).shape

(58492, 14)

If we want to find the distribution of airlines over a range of distances, we need to place the values of the DIST column into discrete bins. Let's use the pandas cut function to split the data into five bins:


In [92]:
bins = [-np.inf, 200, 500, 1000, 2000, np.inf]
cuts = pd.cut(flights['DIST'], bins=bins)
cuts.head()

0     (500.0, 1000.0]
1    (1000.0, 2000.0]
2     (500.0, 1000.0]
3    (1000.0, 2000.0]
4    (1000.0, 2000.0]
Name: DIST, dtype: category
Categories (5, interval[float64, right]): [(-inf, 200.0] < (200.0, 500.0] < (500.0, 1000.0] < (1000.0, 2000.0] < (2000.0, inf]]

An ordered categorical Series is created. To help get an idea of what happened, let's count the values of each category:

In [93]:
cuts.value_counts()

DIST
(500.0, 1000.0]     20659
(200.0, 500.0]      15874
(1000.0, 2000.0]    14186
(2000.0, inf]        4054
(-inf, 200.0]        3719
Name: count, dtype: int64

The cuts Series can now be used to form groups. Pandas allows you to form groups in any way you wish. Pass the cuts Series to the groupby method and then call the value_counts method on the AIRLINE column to find the distribution for each distance group. Notice that SkyWest (OO) makes up 33% of flights less than 200 miles but only 16% of those between 200 and 500 miles:

In [94]:
flights.groupby(cuts)['AIRLINE'].value_counts(normalize=True).round(3).head(15)

C:\Users\Sumanth\AppData\Local\Temp\ipykernel_8552\1498986617.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  flights.groupby(cuts)['AIRLINE'].value_counts(normalize=True).round(3).head(15)


DIST            AIRLINE
(-inf, 200.0]   OO         0.326
                EV         0.289
                MQ         0.211
                DL         0.086
                AA         0.052
                UA         0.027
                WN         0.009
                US         0.000
                AS         0.000
                NK         0.000
                HA         0.000
                F9         0.000
                B6         0.000
                VX         0.000
(200.0, 500.0]  WN         0.194
Name: proportion, dtype: float64

### How it works...

In step 2, the cut function places each value of the DIST column into one of five bins. The bins are created by a sequence of six numbers defining the edges. You always need one more edge than the number of bins. You can pass the bins parameter an integer, which automatically creates that number of equal-width bins. Negative infinity and positive infinity objects are available in NumPy and ensure that all values get placed in a bin. If you have values that are outside of the bin edges, they will be made missing and not be placed in a bin.

The cuts variable is now a Series of five ordered categories. It has all the normal Series methods and in step 3, the value_counts method is used to get a sense of its distribution.

Very interestingly, pandas allows you to pass the groupby method any object. This means that you are able to form groups from something completely unrelated to the current DataFrame. Here, we group by the values in the cuts variable. For each grouping, we find the percentage of flights per airline with value_counts by setting normalize to True.

Some interesting insights can be drawn from this result. Looking at the full result, SkyWest is the leading airline for under 200 miles but has no flights over 2,000 miles. In contrast, American Airlines has the fifth highest total for flights under 200 miles but has by far the most flights between 1,000 and 2,000 miles.

### There's more...

We can find more results when grouping by the cuts variable. For instance, we can find the 25th, 50th, and 75th percentile airtime for each distance grouping. As airtime is in minutes, we can divide by 60 to get hours:



In [95]:
flights.groupby(cuts)['AIR_TIME'].quantile(q=[.25, .5, .75]).div(60).round(2)

C:\Users\Sumanth\AppData\Local\Temp\ipykernel_8552\2062130766.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  flights.groupby(cuts)['AIR_TIME'].quantile(q=[.25, .5, .75]).div(60).round(2)


DIST                  
(-inf, 200.0]     0.25    0.43
                  0.50    0.50
                  0.75    0.57
(200.0, 500.0]    0.25    0.77
                  0.50    0.92
                  0.75    1.05
(500.0, 1000.0]   0.25    1.43
                  0.50    1.65
                  0.75    1.92
(1000.0, 2000.0]  0.25    2.50
                  0.50    2.93
                  0.75    3.40
(2000.0, inf]     0.25    4.30
                  0.50    4.70
                  0.75    5.03
Name: AIR_TIME, dtype: float64

We can use this information to create informative string labels when using the cut function. These labels replace the interval notation. We can also chain the unstack method which transposes the inner index level to column names:

In [96]:
labels=['Under an Hour', '1 Hour', '1-2 Hours', '2-4 Hours', '4+ Hours']
cuts2 = pd.cut(flights['DIST'], bins=bins, labels=labels)
flights.groupby(cuts2)['AIRLINE'].value_counts(normalize=True).round(3).unstack().style.highlight_max(axis=1)

C:\Users\Sumanth\AppData\Local\Temp\ipykernel_8552\4001547007.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  flights.groupby(cuts2)['AIRLINE'].value_counts(normalize=True).round(3).unstack().style.highlight_max(axis=1)


AIRLINE,AA,AS,B6,DL,EV,F9,HA,MQ,NK,OO,UA,US,VX,WN
DIST,,,,,,,,,,,,,,
Under an Hour,0.052000,0.000000,0.000000,0.086000,0.289000,0.000000,0.000000,0.211000,0.000000,0.326000,0.027000,0.000000,0.000000,0.009000
1 Hour,0.071000,0.001000,0.007000,0.189000,0.156000,0.005000,0.000000,0.100000,0.012000,0.159000,0.062000,0.016000,0.028000,0.194000
1-2 Hours,0.144000,0.023000,0.003000,0.206000,0.101000,0.038000,0.000000,0.051000,0.030000,0.106000,0.131000,0.025000,0.004000,0.138000
2-4 Hours,0.264000,0.016000,0.003000,0.165000,0.016000,0.031000,0.000000,0.003000,0.045000,0.046000,0.199000,0.040000,0.012000,0.160000
4+ Hours,0.212000,0.012000,0.080000,0.171000,0.000000,0.004000,0.028000,0.000000,0.019000,0.000000,0.289000,0.065000,0.074000,0.046000


# Counting the total number of flights between cities
In the flights dataset, we have data on the origin and destination airport. It is trivial to count the number of flights originating in Houston and landing in Atlanta, for instance. What is more difficult is counting the total number of flights between the two cities, regardless of which one is the origin or destination.

### Getting ready

In this recipe, we count the total number of flights between two cities regardless of which one is the origin or destination. To accomplish this, we sort the origin and destination airports alphabetically so that each combination of airports always occurs in the same order. We can then use this new column arrangement to form groups and then to count.

### How to do it...

Read in the flights dataset, and find the total number of flights between each origin and destination airport:

In [102]:
flights = pd.read_csv('data/flights.csv')
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


In [103]:
flights_count = flights.groupby(['ORG_AIR', 'DEST_AIR']).size()
flights_count.head()

ORG_AIR  DEST_AIR
ATL      ABE         31
         ABQ         16
         ABY         19
         ACY          6
         AEX         40
dtype: int64

Select the total number of flights between Houston (IAH) and Atlanta (ATL) in both directions:

In [104]:
flights_count.loc[[('ATL', 'IAH'), ('IAH', 'ATL')]]

ORG_AIR  DEST_AIR
ATL      IAH         121
IAH      ATL         148
dtype: int64

We could simply sum these two numbers together to find the total flights between the cities but there is a more efficient and automated solution that can work for all flights. Let's independently sort the origin and destination cities for each row in alphabetical order:

In [106]:
flights_sorted = flights[['ORG_AIR', 'DEST_AIR']].apply(sorted) #, axis=1
flights_sorted.head()

,ORG_AIR,DEST_AIR
0,ATL,ABE
1,ATL,ABE
2,ATL,ABE
3,ATL,ABE
4,ATL,ABE


Now that each row has been independently sorted, the column names are not correct. Let's rename them to something more generic and then again find the total number of flights between all cities:

In [107]:
flights_count_ord = flights_sorted.rename(columns={'ORG_AIR':'AIR1', 'DEST_AIR':'AIR2'}) \
                                  .groupby(['AIR1', 'AIR2']) \
                                  .size()
flights_count_ord.head(10)

AIR1  AIR2
ATL   ABE      55
      ABI      74
      ABQ     343
      ABR      19
      ABY      19
      ACT      49
      ACV      40
      ACY       9
      AEX     109
      AGS      83
dtype: int64

Let's select all the flights between Atlanta and Houston and verify that it matches the sum of the values in step 2:

In [109]:
flights_count_ord.loc[('ATL', 'ABQ')]

343

If we try and select flights with Houston followed by Atlanta, we get an error:


In [110]:
flights_count_ord.loc[('ABQ', 'ATL')]

KeyError: ('ABQ', 'ATL')

### How it works...

In step 1, we form groups by the origin and destination airport columns and then apply the size method to the groupby object, which simply returns the total number of rows for each group. Notice that we could have passed the string size to the agg method to achieve the same result. In step 2, the total number of flights for each direction between Atlanta and Houston are selected. The Series flights_count has a MultiIndex with two levels. One way to select rows from a MultiIndex is to pass the loc indexing operator a tuple of exact level values. Here, we actually select two different rows, ('ATL', 'HOU') and ('HOU', 'ATL'). We use a list of tuples to do this correctly.

Step 3 is the most pertinent step in the recipe. We would like to have just one label for all flights between Atlanta and Houston and so far we have two. If we alphabetically sort each combination of origin and destination airports, we would then have a single label for flights between airports. To do this, we use the DataFrame apply method. This is different from the groupby apply method. No groups are formed in step 3.

The DataFrame apply method must be passed a function. In this case, it's the built-in sorted function. By default, this function gets applied to each column as a Series. We can change the direction of computation by using axis=1 (or axis='index'). The sorted function has each row of data passed to it implicitly as a Series. It returns a list of sorted airport codes. Here is an example of passing the first row as a Series to the sorted function:

In [111]:
sorted(flights.loc[0, ['ORG_AIR', 'DEST_AIR']])

['LAX', 'SLC']

The apply method iterates over all rows using sorted in this exact manner. After completion of this operation, each row is independently sorted. The column names are now meaningless. We rename the column names in the next step and then perform the same grouping and aggregating as was done in step 2. This time, all flights between Atlanta and Houston fall under the same label.

## There's more...
You might be wondering why we can't use the simpler sort_values Series method. This method does not sort independently and instead, preserves the row or column as a single record as one would expect while doing a data analysis. Step 3 is a very expensive operation and takes several seconds to complete. There are only about 60,000 rows so this solution would not scale well to larger data. Calling the

Step 3 is a very expensive operation and takes several seconds to complete. There are only about 60,000 rows so this solution would not scale well to larger data. Calling the apply method with axis=1 is one of the least performant operations in all of pandas. Internally, pandas loops over each row and does not provide any speed boosts from NumPy. If possible, avoid using apply with axis=1.

We can get a massive speed increase with the NumPy sort function. Let's go ahead and use this function and analyze its output. By default, it sorts each row independently:

In [112]:
data_sorted = np.sort(flights[['ORG_AIR', 'DEST_AIR']])
data_sorted[:10]

array([['LAX', 'SLC'],
       ['DEN', 'IAD'],
       ['DFW', 'VPS'],
       ['DCA', 'DFW'],
       ['LAX', 'MCI'],
       ['IAH', 'SAN'],
       ['DFW', 'MSY'],
       ['PHX', 'SFO'],
       ['ORD', 'STL'],
       ['IAH', 'SJC']], dtype=object)

A two-dimensional NumPy array is returned. NumPy does not easily do grouping operations so let's use the DataFrame constructor to create a new DataFrame and check whether it equals the flights_sorted DataFrame from step 3:

In [113]:
flights_sorted2 = pd.DataFrame(data_sorted, columns=['AIR1', 'AIR2'])
flights_sorted2.equals(flights_sorted.rename(columns={'ORG_AIR':'AIR1', 'DEST_AIR':'AIR2'}))

False

As the DataFrames are the same, you can replace step 3 with the previous faster sorting routine. Let's time the difference between each of the different sorting methods:

In [114]:
%timeit flights_sorted = flights[['ORG_AIR', 'DEST_AIR']].apply(sorted, axis=1)

392 ms ± 33.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [115]:
%%timeit
data_sorted = np.sort(flights[['ORG_AIR', 'DEST_AIR']])
flights_sorted2 = pd.DataFrame(data_sorted, columns=['AIR1', 'AIR2'])

9.36 ms ± 343 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


The NumPy solution is an astounding 700 times faster than using apply with panda

# Finding the longest streak of on-time flights
One of the most important metrics for airlines is their on-time flight performance. The Federal Aviation Administration considers a flight delayed when it arrives at least 15 minutes later than its scheduled arrival time. Pandas has direct methods to calculate the total and percentage of on-time flights per airline. While these basic summary statistics are an important metric, there are other non-trivial calculations that are interesting, such as finding the length of consecutive on-time flights for each airline at each of its origin airports.

### Getting ready

In this recipe, we find the longest consecutive streak of on-time flights for each airline at each origin airport. This requires each value in a column to be aware of the value immediately following it. We make clever use of the diff and cumsum methods in order to find streaks before applying this methodology to each of the groups.

### How to do it...

Before we get started with the actual flights dataset, let's practice counting streaks of ones with a small sample Series:

In [116]:
s = pd.Series([1, 1, 1, 0, 1, 1, 1, 0])
s

0    1
1    1
2    1
3    0
4    1
5    1
6    1
7    0
dtype: int64

Our final representation of the streaks of ones will be a Series of the same length as the original with an independent count beginning from one for each streak. To get started, let's use the cumsum method:

In [117]:
s1 = s.cumsum()
s1

0    1
1    2
2    3
3    3
4    4
5    5
6    6
7    6
dtype: int64

We have now accumulated all the ones going down the Series. Let's multiply this Series by the original:

In [118]:
s.mul(s1)

0    1
1    2
2    3
3    0
4    4
5    5
6    6
7    0
dtype: int64

We have only non-zero values where we originally had ones. This result is fairly close to what we desire. We just need to restart each streak at one instead of where the cumulative sum left off. Let's chain the diff method, which subtracts the previous value from the current:

In [119]:
s.mul(s1).diff()

0    NaN
1    1.0
2    1.0
3   -3.0
4    4.0
5    1.0
6    1.0
7   -6.0
dtype: float64

A negative value represents the end of a streak. We need to propagate the negative values down the Series and use them to subtract away the excess accumulation from step 2. To do this, we will make all non-negative values missing with the where method:

In [120]:
s.mul(s1).diff().where(lambda x: x < 0)

0    NaN
1    NaN
2    NaN
3   -3.0
4    NaN
5    NaN
6    NaN
7   -6.0
dtype: float64

We can now propagate these values down with the ffill method:

In [121]:
s.mul(s1).diff().where(lambda x: x < 0).ffill()

0    NaN
1    NaN
2    NaN
3   -3.0
4   -3.0
5   -3.0
6   -3.0
7   -6.0
dtype: float64

Finally, we can add this Series back to s1 to clear out the excess accumulation:

In [122]:
s.mul(s1).diff().where(lambda x: x < 0).ffill().add(s1, fill_value=0)

0    1.0
1    2.0
2    3.0
3    0.0
4    1.0
5    2.0
6    3.0
7    0.0
dtype: float64

Now that we have a working consecutive streak finder, we can find the longest streak per airline and origin airport. Let's read in the flights dataset and create a column to represent on-time arrival:

In [123]:
flights = pd.read_csv('data/flights.csv')
flights['ON_TIME'] = flights['ARR_DELAY'].lt(15).astype(int)
flights[['AIRLINE', 'ORG_AIR', 'ON_TIME']].head(10)

,AIRLINE,ORG_AIR,ON_TIME
0,WN,LAX,0
1,UA,DEN,1
2,MQ,DFW,0
3,AA,DFW,1
4,WN,LAX,0
5,UA,IAH,1
6,AA,DFW,0
7,F9,SFO,1
8,AA,ORD,1
9,UA,IAH,1


Use our logic from the first seven steps to define a function that returns the maximum streak of ones for a given Series:

In [124]:
def max_streak(s):
    s1 = s.cumsum()
    return s.mul(s1).diff().where(lambda x: x < 0) \
            .ffill().add(s1, fill_value=0).max()

Find the maximum streak of on-time arrivals per airline and origin airport along with the total number of flights and percentage of on-time arrivals. First, sort the day of the year and scheduled departure time:

In [125]:
flights.sort_values(['MONTH', 'DAY', 'SCHED_DEP']) \
       .groupby(['AIRLINE', 'ORG_AIR'])['ON_TIME'] \
       .agg(['mean', 'size', max_streak]).round(2).head()

mean  size  max_streak
AIRLINE ORG_AIR                        
AA      ATL      0.82   233        15.0
        DEN      0.74   219        17.0
        DFW      0.78  4006        64.0
        IAH      0.80   196        24.0
        LAS      0.79   374        29.0

### How it works...

Finding streaks in the data is not a straightforward operation in pandas and requires methods that look ahead or behind, such as diff or shift, or those that remember their current state, such as cumsum. The final result from the first seven steps is a Series the same length as the original that keeps track of all consecutive ones. Throughout these steps, we use the mul and add methods instead of their operator equivalents (*) and (+). In my opinion, this allows for a slightly cleaner progression of calculations from left to right. You, of course, can replace these with the actual operators.

Ideally, we would like to tell pandas to apply the cumsum method to the start of each streak and reset itself after the end of each one. It takes many steps to convey this message to pandas. Step 2 accumulates all the ones in the Series as a whole. The rest of the steps slowly remove any excess accumulation. In order to identify this excess accumulation, we need to find the end of each streak and subtract this value from the beginning of the next streak.

To find the end of each streak, we cleverly make all values not part of the streak zero by multiplying s1 by the original Series of zeros and ones in step 3. The first zero following a non-zero, marks the end of a streak. That's good, but again, we need to eliminate the excess accumulation. Knowing where the streak ends doesn't exactly get us there.

In step 4, we use the diff method to find this excess. The diff method takes the difference between the current value and any value located at a set number of rows away from it. By default, the difference between the current and the immediately preceding value is returned.

Only negative values are meaningful in step 4. Those are the ones immediately following the end of a streak. These values need to be propagated down until the end of the following streak. To eliminate (make missing) all the values we don't care about, we use the where method, which takes a Series of conditionals of the same size as the calling Series. By default, all the True values remain the same, while the False values become missing. The where method allows you to use the calling Series as part of the conditional by taking a function as its first parameter. An anonymous function is used, which gets passed the calling Series implicitly and checks whether each value is less than zero. The result of step 5 is a Series where only the negative values are preserved with the rest changed to missing.

The ffill method in step 6 replaces missing values with the last non-missing value going forward/down a Series. As the first three values don't follow a non-missing value, they remain missing. We finally have our Series that removes the excess accumulation. We add our accumulation Series to the result of step 6 to get the streaks all beginning from zero. The add method allows us to replace the missing values with the fill_value parameter. This completes the process of finding streaks of ones in the dataset. When doing complex logic like this, it is a good idea to use a small dataset where you know what the final output will be. It would be quite a difficult task to start at step 8 and build this streak-finding logic while grouping.

In step 8, we create the ON_TIME column. One item of note is that the cancelled flights have missing values for ARR_DELAY, which do not pass the boolean condition and therefore result in a zero for the ON_TIME column. Canceled flights are treated the same as delayed.

Step 9 turns our logic from the first seven steps into a function and chains the max method to return the longest streak. As our function returns a single value, it is formally an aggregating function and can be passed to the agg method as done in step 10. To ensure that we are looking at actual consecutive flights, we use the sort_values method to sort by date and scheduled departure time.
